<a href="https://colab.research.google.com/github/jyryu3161/lecture/blob/main/Docking_%EC%8B%A4%EC%8A%B5_multiple_molecules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Docking with Smina 

This Google Colab notebook is inspired from [the post of **Esben Jannik Bjerrum** about using smina for docking](https://www.wildcardconsulting.dk/ligand-docking-with-smina/)

In [2]:
# download & extract
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
# move to python packages directory
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.6/dist-packages/
x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
# rdkit need libboost_python3.so.1.65.1
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3995    0  3995    0     0  19975      0 --:--:-- --:--:-- --:--:-- 19975
100 20.2M  100 20.2M    0     0  5129k      0  0:00:04  0:00:04 --:--:-- 6732k


In [3]:
!git clone https://github.com/jyryu3161/lecture.git
!chmod u+x ./lecture/qvina2.1

Cloning into 'lecture'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [19]:
!pip install --pre deepchem
!pip install ogb
!pip install py3Dmol # 3D Molecular Visualizer
!pip install oddt
!pip install -U ProDy
!apt-get install pymol 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 16.0 MB/s 
  Created wheel for oddt: filename=oddt-0.5-py2.py3-none-any.whl size=1538781 sha256=a349868f40587cdd0e937ad9145a286049345c53dd8a5a135c5fe05d772f4af1
  Stored in directory: /root/.cache/pip/wheels/cc/2c/c7/782df4a53302f120d5b27d1323e90f7f2d370acefd6e50180d
Successfully built oddt
  Attempting uninstall: oddt
    Found existing installation: oddt 0.7
    Uninstalling oddt-0.7:
      Successfully uninstalled oddt-0.7


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
pymol is already the newest version (1.8.4.0+dfsg-1build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [5]:
import py3Dmol
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

def make_3d_coordinate(m):
    target_mols = []
    m = Chem.AddHs(m)
    potential = AllChem.ETKDG()
    AllChem.EmbedMolecule(m, potential)
    cids = AllChem.EmbedMultipleConfs(m, numConfs=1, numThreads=15)
    
    for conf in m.GetConformers():
        tm = Chem.Mol(m,False,conf.GetId())
        res = AllChem.MMFFOptimizeMoleculeConfs(tm)
        target_mols.append(tm)
    return target_mols[0]

def drawit2(m,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p

def DrawComplex(protein,ligand):
    complex_pl = Chem.MolToPDBBlock(Chem.CombineMols(protein,ligand))
    view = py3Dmol.view(width=600,height=600)
    view.addModel(complex_pl,'pdb')
    chA = {'chain':['H','L','I']}
    chB = {'resn':'UNL'}
    view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
    #view.setStyle(chA,{'lines': {}})
    view.addSurface(py3Dmol.VDW,{'opacity':0.8}, chB)
    view.setStyle(chB,{'stick':{}})
    view.zoomTo()
    return view   


In [13]:
from prody import *
import os

pdb_id = '6M0K'
ligand_Id = 'FJC'
compound_file = './input_smiles.txt'

docking_output_dir = './output/'
tmp_mol_dir = './output/molecules/'
tmp_output_dir = './output/docking_results/'

try:
    os.mkdir(docking_output_dir)
except:
    pass

try:
    os.mkdir(tmp_mol_dir)
except:
    pass

try:
    os.mkdir(tmp_output_dir)
except:
    pass

pdb = parsePDB(pdb_id)

protein = pdb.select('protein')
ligand = pdb.select('resname %s'%(ligand_Id))

writePDB('protein.pdb', protein)
writePDB('ligand.pdb', ligand)


@> PDB file is found in working directory (6m0k.pdb.gz).
DEBUG:.prody:PDB file is found in working directory (6m0k.pdb.gz).
@> 2560 atoms and 1 coordinate set(s) were parsed in 0.05s.
DEBUG:.prody:2560 atoms and 1 coordinate set(s) were parsed in 0.05s.


'ligand.pdb'

In [12]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem


with open('./input_smiles.txt', 'r') as fp:
    for line in fp:
        sptlist = line.strip().split('\t')
        compound_id = sptlist[0].strip()
        smiles = sptlist[1].strip()

        mol = Chem.MolFromSmiles(smiles)    
        mol = Chem.AddHs(mol)
        mol = make_3d_coordinate(mol)

        with Chem.SDWriter(tmp_mol_dir+'%s.sdf'%(compound_id)) as w:
            w.write(mol)


In [17]:
import oddt
from oddt.docking.AutodockVina import autodock_vina
from oddt.docking import AutodockVina

import os
import glob

protein_file = './protein.pdb'
ligand_file = './ligand.pdb'

protein = next(oddt.toolkit.readfile('pdb', protein_file))
protein.protein = True
ligand = next(oddt.toolkit.readfile('pdb', ligand_file))

files = glob.glob(tmp_mol_dir+'/*.sdf')
for each_file in files:
    basename = os.path.basename(each_file).split('.')[0].strip()
    mol = Chem.MolFromMolFile(each_file)

    mol2 = Chem.MolToMolBlock(mol)
    dock_mol = oddt.toolkit.readstring('sdf', mol2)

    vina_obj = autodock_vina(protein=protein, auto_ligand=ligand, size=(20, 20, 20), num_modes=1, exhaustiveness=8, executable='./lecture/qvina2.1')

    docking_outputs = vina_obj.dock([dock_mol])
    for each_output in docking_outputs:
        vina_affinity = each_output.data['vina_affinity']
        AutodockVina.write_vina_pdbqt(each_output, tmp_output_dir, name_id='best_docking_pose_%s'%(basename))
        
        print ("COMPOUND:%s, binding energy (kcal/mol) : %s"%(basename, vina_affinity))
        break




COMPOUND:BRD-K01825000-001-02-9, binding energy (kcal/mol) : -4.2
COMPOUND:BRD-K00004970-001-01-9, binding energy (kcal/mol) : -7.6
COMPOUND:BRD-K76022557-003-28-9, binding energy (kcal/mol) : -6.6
COMPOUND:BRD-A29260609-003-26-0, binding energy (kcal/mol) : -5.3
COMPOUND:BRD-K50071428-001-04-9, binding energy (kcal/mol) : -7.5
COMPOUND:BRD-K64034691-001-01-9, binding energy (kcal/mol) : -7.0
COMPOUND:BRD-K26262077-238-01-5, binding energy (kcal/mol) : -3.7
COMPOUND:BRD-K44276885-001-06-6, binding energy (kcal/mol) : -7.1
COMPOUND:BRD-K33622447-066-01-9, binding energy (kcal/mol) : -7.7
COMPOUND:BRD-K24048528-001-03-9, binding energy (kcal/mol) : -7.1


In [18]:
filename = tmp_output_dir+'best_docking_pose_BRD-K01825000-001-02-9.pdbqt'

ligand = next(oddt.toolkit.readfile('pdbqt', filename))

protein = Chem.MolFromPDBFile('protein.pdb')
DrawComplex(protein, ligand.Mol)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol